In [1]:
import os
import hopsworks
import polars as pl

In [ ]:
from utils.feature_engineering import get_latest_sold_products
get_latest_sold_products()[0]

In [ ]:
with open('data/hopsworks-api-key.txt', 'r') as file:
    os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()
    
project = hopsworks.login()

## Get data

### Quick data exploration
Taken from https://graildient-descent.streamlit.app/eda, using 10,000 samples.
We could do the same using the larger dataset.

#### Numerical/quantitative features
Target variable (sold_price):
- Most sold items are between 35-135$ (consider plotting bins) - consider outliers, since we are far from normal distribution, maybe try a log transformation?

Number of photos:
- Another numerical feature, could be added
- Price increases until 13 photos, then inconsistent

#### Categorical/qualitative features
- high-cardinality: designer, color, subcategory, size
- low-cardinality: category, condition

Target encoding more fitting for high-cardinality features. Whereas low-cardinality features could be one-hot encoded.

- designer a strong predictor
- we skip department, to focus on men's clothing only (it has better representation)
- there is good variation of sold prices in different subcategories => probably a good indicator
- could be interesting to use embeddings for color instead
- condition - perfect for ordinal encoding. The better the condition, the higher the average sold price.

##### Text
- title/description - should we do any pre-processing?
- we could look at sentiment and similar text analysis approaches

#### Images
- could be interesting to add embedding representation of title image

## Feature processing

In [4]:
from utils.feature_engineering import pipeline
df = pipeline(no_of_hits=300)
df.shape

The model BAAI/bge-small-en-v1.5 is ready to use.
The model BAAI/bge-small-en-v1.5 is ready to use.
The model BAAI/bge-small-en-v1.5 is ready to use.
The model BAAI/bge-small-en-v1.5 is ready to use.
shape: (296, 13)
┌──────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ id       ┆ sold_at   ┆ designer_ ┆ descripti ┆ … ┆ color     ┆ followern ┆ sold_pric ┆ embedded_ │
│ ---      ┆ ---       ┆ names     ┆ on        ┆   ┆ ---       ┆ o         ┆ e         ┆ hashtags  │
│ i64      ┆ datetime[ ┆ ---       ┆ ---       ┆   ┆ str       ┆ ---       ┆ ---       ┆ ---       │
│          ┆ μs]       ┆ object    ┆ object    ┆   ┆           ┆ i64       ┆ i64       ┆ object    │
╞══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 69438334 ┆ 2025-01-0 ┆ [-1.96712 ┆ [-3.01418 ┆ … ┆ beige     ┆ 1         ┆ 95        ┆ [-2.87136 │
│          ┆ 5 11:00:1 ┆ 501e-02   ┆ 602e-02   ┆   ┆           ┆           ┆

(296, 13)

## Save data

In [10]:
fs = project.get_feature_store() 

In [11]:
grailed_items_fg = fs.get_or_create_feature_group(
    name='grailed_items',
    description='Sold Grailed items',
    version=2,
    primary_key=['id'],
    event_time="sold_at",
    # expectation_suite=aq_expectation_suite
)

In [ ]:
grailed_items_fg.insert(df)

In [ ]:
# TODO: Update feature description